In [2]:
import pyspark
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, when
# from pyspark.sql.types import types

In [3]:
spark = SparkSession.builder.appName('TesteDataframe').getOrCreate()
df = spark.read.load('D:/donwloads/wc2018-players.csv', header=True, inferSchema=True, sep=',', format='csv')
df.show()

+---------+---+----+------------------+----------+----------+--------------------+------+------+
|     Team|  #|Pos.| FIFA Popular Name|Birth Date|Shirt Name|                Club|Height|Weight|
+---------+---+----+------------------+----------+----------+--------------------+------+------+
|Argentina|  3|  DF|TAGLIAFICO Nicolas|31.08.1992|TAGLIAFICO|      AFC Ajax (NED)|   169|    65|
|Argentina| 22|  MF|    PAVON Cristian|21.01.1996|     PAVÓN|CA Boca Juniors (...|   169|    65|
|Argentina| 15|  MF|    LANZINI Manuel|15.02.1993|   LANZINI|West Ham United F...|   167|    66|
|Argentina| 18|  DF|    SALVIO Eduardo|13.07.1990|    SALVIO|    SL Benfica (POR)|   167|    69|
|Argentina| 10|  FW|      MESSI Lionel|24.06.1987|     MESSI|  FC Barcelona (ESP)|   170|    72|
|Argentina|  4|  DF|  ANSALDI Cristian|20.09.1986|   ANSALDI|     Torino FC (ITA)|   181|    73|
|Argentina|  5|  MF|      BIGLIA Lucas|30.01.1986|    BIGLIA|      AC Milan (ITA)|   175|    73|
|Argentina|  7|  MF|       BAN

### Pega todas as colunas do Dataframe, e coloca elas em uma posição visivel para eu alterar posteriormente

In [4]:
colunas = {
    col:col 
    for col in df.columns
}
display(colunas)

{'Team': 'Team',
 '#': '#',
 'Pos.': 'Pos.',
 'FIFA Popular Name': 'FIFA Popular Name',
 'Birth Date': 'Birth Date',
 'Shirt Name': 'Shirt Name',
 'Club': 'Club',
 'Height': 'Height',
 'Weight': 'Weight'}

### Trocando o nomes das colunas com dois metodos

In [5]:
# Primeiro método - Muito mais inteligente, copio e colo as colunas acima e depois só substituo uma vez
colunas = {
    'pos.': 'POSICAO',
    'FIFA Popular Name': 'NOME_FIFA',
    'Shirt Name': 'NOME_JOGADOR',
    'CLUB': 'CLUBE',
    'Birth Date': 'DT_NASCIMENTO',
    'Height': 'ALTURA_JOGADOR',
    'Weight': 'PESO',
    'Team': 'TIME',
    '#': 'ID_JOGADOR'
}

for nome_antigo, novo_nome in colunas.items():
    df = df.withColumnRenamed(nome_antigo, novo_nome)


# Segundo método - Feito Totalmente Manualmente. 
df = df.withColumnRenamed('pos.','POSICAO')
df = df.withColumnRenamed('FIFA Popular Name', 'NOME_FIFA')
df = df.withColumnRenamed('Shirt Name', 'NOME_JOGADOR')
df = df.withColumnRenamed('CLUB', 'CLUBE')
df = df.withColumnRenamed('Birth Date', 'DT_NASCIMENTO')
df = df.withColumnRenamed('Height', 'ALTURA_JOGADOR')
df = df.withColumnRenamed('Weight', 'PESO')
df = df.withColumnRenamed('Team', 'TIME')
df = df.withColumnRenamed('#', 'ID_JOGADOR')
df.show()

+---------+----------+-------+------------------+-------------+------------+--------------------+--------------+----+
|     TIME|ID_JOGADOR|POSICAO|         NOME_FIFA|DT_NASCIMENTO|NOME_JOGADOR|               CLUBE|ALTURA_JOGADOR|PESO|
+---------+----------+-------+------------------+-------------+------------+--------------------+--------------+----+
|Argentina|         3|     DF|TAGLIAFICO Nicolas|   31.08.1992|  TAGLIAFICO|      AFC Ajax (NED)|           169|  65|
|Argentina|        22|     MF|    PAVON Cristian|   21.01.1996|       PAVÓN|CA Boca Juniors (...|           169|  65|
|Argentina|        15|     MF|    LANZINI Manuel|   15.02.1993|     LANZINI|West Ham United F...|           167|  66|
|Argentina|        18|     DF|    SALVIO Eduardo|   13.07.1990|      SALVIO|    SL Benfica (POR)|           167|  69|
|Argentina|        10|     FW|      MESSI Lionel|   24.06.1987|       MESSI|  FC Barcelona (ESP)|           170|  72|
|Argentina|         4|     DF|  ANSALDI Cristian|   20.0

### Selecionando Dados, como se fosse consultas do SQL

In [8]:
# Selecionando colunas
# from pyspark.sql.functions import select
df_select = df.select( 
    col('TIME').alias('Pais')
    ,col('NOME_JOGADOR') 
    ,col('CLUBE')
)
df_select.show()

+---------+------------+--------------------+
|     Pais|NOME_JOGADOR|               CLUBE|
+---------+------------+--------------------+
|Argentina|  TAGLIAFICO|      AFC Ajax (NED)|
|Argentina|       PAVÓN|CA Boca Juniors (...|
|Argentina|     LANZINI|West Ham United F...|
|Argentina|      SALVIO|    SL Benfica (POR)|
|Argentina|       MESSI|  FC Barcelona (ESP)|
|Argentina|     ANSALDI|     Torino FC (ITA)|
|Argentina|      BIGLIA|      AC Milan (ITA)|
|Argentina|      BANEGA|    Sevilla FC (ESP)|
|Argentina|  MASCHERANO|Hebei China Fortu...|
|Argentina|      DYBALA|   Juventus FC (ITA)|
|Argentina|      AGÜERO|Manchester City F...|
|Argentina|     HIGUAÍN|   Juventus FC (ITA)|
|Argentina|    DI MARÍA|Paris Saint-Germa...|
|Argentina|    LO CELSO|Paris Saint-Germa...|
|Argentina|        MEZA|CA Independiente ...|
|Argentina|       ACUÑA|   Sporting CP (POR)|
|Argentina|   CABALLERO|    Chelsea FC (ENG)|
|Argentina|     MERCADO|    Sevilla FC (ESP)|
|Argentina|    OTAMENDI|Manchester

### Filtrar DataFrame para mostrar os pontos onde não tenha argentina

In [20]:
# porque o comando abaixo não funciona ?
df_filtrado = df.filter( col('TIME') != "Argentina").show()

+---------+----------+-------+------------------+-------------+------------+--------------------+--------------+----+
|     TIME|ID_JOGADOR|POSICAO|         NOME_FIFA|DT_NASCIMENTO|NOME_JOGADOR|               CLUBE|ALTURA_JOGADOR|PESO|
+---------+----------+-------+------------------+-------------+------------+--------------------+--------------+----+
|Australia|        16|     DF|       BEHICH Aziz|   16.12.1990|      BEHICH|     Bursaspor (TUR)|           170|  63|
|Australia|        10|     FW|      KRUSE Robbie|   05.10.1988|       KRUSE|    VfL Bochum (GER)|           180|  66|
|Australia|        19|     DF|     RISDON Joshua|   27.07.1992|      RISDON|WS Wanderers FC (...|           169|  70|
|Australia|         3|     DF|    MEREDITH James|   05.04.1988|    MEREDITH|   Millwall FC (ENG)|           179|  71|
|Australia|        13|     MF|        MOOY Aaron|   15.09.1990|        MOOY|Huddersfield Town...|           173|  72|
|Australia|        14|     FW|    MacLAREN Jamie|   29.0

### FILTRANDO DATAFRAME COM CONDIÇÕES AND e OR

In [31]:
display('DF CONDIÇÃO AND')
# FILTRANDO USANDO A CONDIÇÃO AND
df_filtrado_e = df.filter( 
    (col('TIME') == 'Brazil') 
    & (col('POSICAO') == 'DF')
    & (col('ALTURA_JOGADOR') >= '180')
).show()

display('DF CONDIÇÃO OR')
# FILTRANDO USANDO A CONDIÇÃO OR
df_filtrado_e = df.filter( 
    (col('TIME') == 'Brazil') 
    & (col('POSICAO') == 'DF')
    & ( (col('ALTURA_JOGADOR') == '186') | (col('ALTURA_JOGADOR') == '183'))
).show()


'DF CONDIÇÃO AND'

+------+----------+-------+-------------+-------------+------------+--------------------+--------------+----+
|  TIME|ID_JOGADOR|POSICAO|    NOME_FIFA|DT_NASCIMENTO|NOME_JOGADOR|               CLUBE|ALTURA_JOGADOR|PESO|
+------+----------+-------+-------------+-------------+------------+--------------------+--------------+----+
|Brazil|         6|     DF|  FILIPE LUIS|   09.08.1985| FILIPE LUIS|Atletico Madrid (...|           182|  73|
|Brazil|        13|     DF|   MARQUINHOS|   14.05.1994|  MARQUINHOS|Paris Saint-Germa...|           183|  75|
|Brazil|         3|     DF|      MIRANDA|   07.09.1984|     MIRANDA|FC Internazionale...|           186|  78|
|Brazil|        14|     DF|       DANILO|   15.07.1991|      DANILO|Manchester City F...|           184|  78|
|Brazil|         2|     DF| THIAGO SILVA|   22.09.1984|    T. SILVA|Paris Saint-Germa...|           183|  79|
|Brazil|         4|     DF|PEDRO GEROMEL|   21.09.1985|     GEROMEL|   Grêmio FBPA (BRA)|           190|  84|
+------+--

'DF CONDIÇÃO OR'

+------+----------+-------+------------+-------------+------------+--------------------+--------------+----+
|  TIME|ID_JOGADOR|POSICAO|   NOME_FIFA|DT_NASCIMENTO|NOME_JOGADOR|               CLUBE|ALTURA_JOGADOR|PESO|
+------+----------+-------+------------+-------------+------------+--------------------+--------------+----+
|Brazil|        13|     DF|  MARQUINHOS|   14.05.1994|  MARQUINHOS|Paris Saint-Germa...|           183|  75|
|Brazil|         3|     DF|     MIRANDA|   07.09.1984|     MIRANDA|FC Internazionale...|           186|  78|
|Brazil|         2|     DF|THIAGO SILVA|   22.09.1984|    T. SILVA|Paris Saint-Germa...|           183|  79|
+------+----------+-------+------------+-------------+------------+--------------------+--------------+----+



In [59]:
from pyspark.sql.functions import *

# PEGA AS PRIMEIRAS TRÊS LETRAS DA SELEÇÃO
df_nova_coluna = df.withColumn('tres_primeiras_letras', substring(col('TIME'), 0, 3)).show()

+---------+----------+-------+------------------+-------------+------------+--------------------+--------------+----+---------------------+
|     TIME|ID_JOGADOR|POSICAO|         NOME_FIFA|DT_NASCIMENTO|NOME_JOGADOR|               CLUBE|ALTURA_JOGADOR|PESO|tres_primeiras_letras|
+---------+----------+-------+------------------+-------------+------------+--------------------+--------------+----+---------------------+
|Argentina|         3|     DF|TAGLIAFICO Nicolas|   31.08.1992|  TAGLIAFICO|      AFC Ajax (NED)|           169|  65|                  Arg|
|Argentina|        22|     MF|    PAVON Cristian|   21.01.1996|       PAVÓN|CA Boca Juniors (...|           169|  65|                  Arg|
|Argentina|        15|     MF|    LANZINI Manuel|   15.02.1993|     LANZINI|West Ham United F...|           167|  66|                  Arg|
|Argentina|        18|     DF|    SALVIO Eduardo|   13.07.1990|      SALVIO|    SL Benfica (POR)|           167|  69|                  Arg|
|Argentina|        1

In [60]:
# PEGA O ANO DE NASCIMENTO PARTINDO DAS 4 ULTIMAS LETRAS
df_nova_coluna2 = df.withColumn('ANO', substring('DT_NASCIMENTO', -4, 4)).show()

+---------+----------+-------+------------------+-------------+------------+--------------------+--------------+----+----+
|     TIME|ID_JOGADOR|POSICAO|         NOME_FIFA|DT_NASCIMENTO|NOME_JOGADOR|               CLUBE|ALTURA_JOGADOR|PESO| ANO|
+---------+----------+-------+------------------+-------------+------------+--------------------+--------------+----+----+
|Argentina|         3|     DF|TAGLIAFICO Nicolas|   31.08.1992|  TAGLIAFICO|      AFC Ajax (NED)|           169|  65|1992|
|Argentina|        22|     MF|    PAVON Cristian|   21.01.1996|       PAVÓN|CA Boca Juniors (...|           169|  65|1996|
|Argentina|        15|     MF|    LANZINI Manuel|   15.02.1993|     LANZINI|West Ham United F...|           167|  66|1993|
|Argentina|        18|     DF|    SALVIO Eduardo|   13.07.1990|      SALVIO|    SL Benfica (POR)|           167|  69|1990|
|Argentina|        10|     FW|      MESSI Lionel|   24.06.1987|       MESSI|  FC Barcelona (ESP)|           170|  72|1987|
|Argentina|     

### Conversão de tipos de variaveis. 

In [80]:
schema = df.schema
tipos_dados_colunas = {col.name: col.dataType for col in schema}

# Imprimir os tipos de colunas
for nome_coluna, tipo_coluna in tipos_dados_colunas.items():
    print(f"'{nome_coluna}': {tipo_coluna},")

'TIME': StringType(),
'ID_JOGADOR': IntegerType(),
'POSICAO': StringType(),
'NOME_FIFA': StringType(),
'DT_NASCIMENTO': StringType(),
'NOME_JOGADOR': StringType(),
'CLUBE': StringType(),
'ALTURA_JOGADOR': IntegerType(),
'PESO': IntegerType(),


In [83]:
# Primeira Forma: Copio e colo as variáveis e seus respectivos jeitos para automatizar
# e visualizar todas que precisarei trocar os tipos
from pyspark.sql.types import * 

colt = {
'TIME': StringType(),
'ID_JOGADOR': StringType(),
'POSICAO': StringType(),
'NOME_FIFA': StringType(),
'DT_NASCIMENTO': StringType(),
'NOME_JOGADOR': StringType(),
'CLUBE': StringType(),
'ALTURA_JOGADOR': IntegerType(),
'PESO': IntegerType(),
}

# Aplicar a conversão de tipos
for chave, valor in colt.items():
    df = df.withColumn(chave, col(chave).cast(valor))



# SEGUNDA FORMA: Troco os tipos manualmente
df = df.withColumn('ALTURA_JOGADOR', col('ALTURA_JOGADOR').cast(StringType()))
df = df.withColumn('PESO', col('PESO').cast(StringType()))
df.printSchema()

root
 |-- TIME: string (nullable = true)
 |-- ID_JOGADOR: string (nullable = true)
 |-- POSICAO: string (nullable = true)
 |-- NOME_FIFA: string (nullable = true)
 |-- DT_NASCIMENTO: string (nullable = true)
 |-- NOME_JOGADOR: string (nullable = true)
 |-- CLUBE: string (nullable = true)
 |-- ALTURA_JOGADOR: string (nullable = true)
 |-- PESO: string (nullable = true)

